In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 64 # what is the maximum context length for predictions?
epochs = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 8
dropout = 0.0

In [27]:
import json
# load json file alex.json
with open('alex.json', encoding="utf-8") as f:
    data = json.load(f)

In [28]:
# convert all message contents into a single string
text = '\n'.join([f"{m['author']['name']}:{m['content']}" for m in data["messages"]])
# write text to a file alex.txt
with open('alex.txt', 'w', encoding="utf-8") as f:
    f.write(text)


In [39]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

torch.Size([769508])


In [7]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [33]:
sp = spm.SentencePieceProcessor()
sp.load('alex.model')

True

In [41]:
data = sp.encode_as_ids(text)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [10]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [11]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


In [12]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [13]:
# super simple transformer model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [14]:
model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

0.426038 M parameters


In [15]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
import wandb
run = wandb.init(
    project="nanogpt-testing",
    config={
        "learning_rate": learning_rate,
        "epochs": epochs,
        "batch_size": batch_size,
        "n_layer": n_layer,
        "n_head": n_head,
        "n_embd": n_embd,
        "block_size": block_size,}
)
for iter in range(epochs):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == epochs - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    wandb.log({"loss": loss})
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

run.finish()

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


OpTiMiStC:I'm different will bvpnilonitys one
Medori:actuallyswogde
Medori:2 minutuubled'>3 calouse numbers
Medori:Pine detet
OpTiMiStC:Yeah
Medori:i love started using by life
Medori:what is a rn
Medori:Yh
Medori:Imellist that pain colla
Medori:Got nice assk time just fectly year
OpTiMiStC:Aprop for atlinor
OpTiMiStC:Just zAV you sto composy just mum at the worstly ethan
OpTiMiStC:cody is doing
OpTiMiStC:is the same
OpTiMiStC:when we home
OpTiMiStC:We don't her swead I cum not gonna uplot scores
OpTiMiStC:You could
OpTiMiStC:Honestly give a fever
Medori:When u think it
OpTiMiStC:Like autoi rlly remember,1 you play sent apposible in a diffebul littling
Medori:I was finitially spit network chemistry
OpTiMiStC:i be thinking under u get complity an english time to pretty into are a clips the so chem
OpTiMiStC:
Medori:oh yh way way tmny for midy print
Medori:huhh
Medori:How
OpTiMiStC:https://youtu.be/V7q0k5A
OpTiMiStC:
OpTiMiStC:oh I start this was not much by arough
OpTiMiStC:bro that is

In [ ]:
# save the model
torch.save(model.state_dict(), 'bigmodel.pt')

# import the model

In [9]:
class MultiHeadAttention(nn.Module):

    def __init__(self):
        super().__init__()
        self.MHA = nn.MultiheadAttention(n_embd, n_head, dropout=dropout)
        

    def forward(self, x):
        B,T,C = x.shape
        self.MHA(x,x,x)
        
        return x

In [17]:
MultiHeadAttention().forward(torch.randn(32, 64, 64))

tensor([[[ 7.5387e-01, -9.0065e-01, -1.2124e+00,  ..., -6.6374e-01,
          -1.0894e+00, -5.8394e-01],
         [-1.0988e+00, -3.0963e-01, -1.4654e+00,  ..., -9.4405e-01,
           1.6175e+00,  1.5655e+00],
         [ 4.5148e-01, -1.3236e+00,  2.4697e-02,  ..., -4.9402e-01,
           6.2258e-02,  1.1916e+00],
         ...,
         [-3.5254e-01, -1.0588e+00,  1.2127e+00,  ..., -2.6041e-01,
           3.1462e-01, -2.7874e-01],
         [-1.5985e+00,  7.2194e-01,  6.2852e-01,  ...,  1.1886e+00,
           7.7472e-02,  4.6995e-01],
         [-3.2228e-01,  9.5099e-01, -2.0557e+00,  ...,  6.7635e-01,
           1.0454e+00, -1.1562e-01]],

        [[ 2.0435e-01, -2.9111e-01, -7.0913e-01,  ..., -5.2609e-01,
          -4.4271e-01, -2.8485e-01],
         [-1.4839e+00,  8.5877e-01, -7.2787e-02,  ...,  1.8995e+00,
          -1.4642e-01,  2.2801e-01],
         [-7.6403e-01, -1.1333e+00, -3.4127e-01,  ..., -7.3189e-01,
          -3.0177e-01,  3.3670e-02],
         ...,
         [-1.4567e-01,  7